In [1]:
from bs4 import BeautifulSoup
import pandas as pd
#import urllib2
import csv

In [2]:
def assignNoLabelStatus(s):
    status = "Label"
    s = s.strip(',').split(',')
    t = s[0]
    if len(t) > 1:
        # case to discard, eg C123G
        if t[0] in list('ACGT') and t[-1] in list('ACGT'):
            status = "NoLabel"
        # case to discard, eg C123G!
        elif t[0] in list('ACGT') and t[-1] == "!":
            status = "NoLabel"
        elif t[0].isalpha() == False:
            status = "NoLabel"
        # This last case replaces the next ones
        # case to discard, eg deletion 8123-8234d
        #elif t[0].isdigit() and t[-1] == "d":
        #    status = "NoLabel"
        # eg (C64T)
        #elif t[0] == "(":
        #    status = "NoLabel"
    return status

def checkIfAC(s):
    if len(s) > 0:
        if s[0].isalpha() and s[-1].isdigit():
            return True
        else:
            return False
    else:
        return False

In [3]:
#url = 'http://www.data.jma.go.jp/obd/stats/etrn/view/monthly_s3_en.php?block_no=47401&view=1'
#html = urllib2.urlopen(url).read()
html = open('mtDNA_tree_Build_17.onlytable.htm', 'r').read()
#print(repr(html))

In [4]:
soup = BeautifulSoup(html, 'lxml')

In [5]:
table = soup.find_all("table")[0]

In [6]:
rows = table.find_all("tr")

In [8]:
# col_normal = rows[6]

In [9]:
csv_list = []
o = open('phylotree_17_190121.csv', 'w')
for row in rows:
    s = ""
    columns = row.find_all('td')
    for column in columns:
        column_text = ','.join(column.get_text().split())
        s += "{},".format(column_text)
        s = s.replace("\A0", "")
    if s.strip(',') != "":
        #print(s)
        #print(s.rstrip(','))
        s_list = s.rstrip(',').split(',')#[:-1]
        #print(s_list)
        # Check if it's a NoLabel
        for x,i in enumerate(s_list):
            if i != "":
                if i == 'L0a1a' or i == "L0":
                    print(s)
                status = assignNoLabelStatus(s)
                if status == "NoLabel":
                    #print("Found NoLabel at {}".format(','.join(s_list)))
                    s_list[x-1] = status
                    break
        # Delete ACs at the end
        #print(s_list)
        #try:
        while checkIfAC(s_list[-1]) == True:
            #print("Gonna reduce this: {}".format(s_list))
            s_list = s_list[:-1]
        while s_list[-1] == "":
            s_list = s_list[:-1]
        #except:
#            print("Error for:\n{}".format(s_list))
            #print(s_list)
        s = ','.join(s_list)
        if status == "NoLabel":
            print s
        csv_list.append(s)
        o.write(s+"\n")
o.close()

L0,G263A,C1048T,C3516a,T5442C,T6185C,C9042T,A9347G,G10589A,G12007A,A12720G,,,,,,,,,,,,,,,,,,,
,,,,,,,,L0a1a,(C64T),T3866C,,,,,,,,,,,,,,,EU092665,EU092764,
,,,,,,,,,NoLabel,A200G
,,,,,,,,NoLabel,A16293G
,,,,,,NoLabel,C152T
,,,,,NoLabel,A189G
,,,,,,,,NoLabel,T198C!
,,,,,,,,NoLabel,T16189C!,(C16192T)
,,,,,,,,,,NoLabel,G143A
,,,,,,,,NoLabel,G143A
,,,,,,,,,,NoLabel,T16086C
,,,,,,,,,,NoLabel,G16129A!
,,,,,,,,,NoLabel,G16309A!
,,,,,,,,,NoLabel,T16189C!,(C16192T)
,,,,,,,,,,NoLabel,G16309A!
,,,,,,,NoLabel,G16129A!
,,,,,,,NoLabel,G709A
,,,,,,,,,,NoLabel,T152C!
,,,,,,,,,,NoLabel,C16124T!
,,,,,,,,,,NoLabel,C16292T
,,,,,,,,,,,NoLabel,C150T
,,,,,,,,,,NoLabel,T152C!
,,,,,,,,,,NoLabel,T195C!
,,,,,,,,,NoLabel,T16311C!
,,,,,,,,,,,NoLabel,T16093C
,,,,,,,,,,,,NoLabel,G207A
,,,,,,,,,,,,,,NoLabel,T16093C
,,,,,,,,,,NoLabel,T204C
,,,,,,,,,NoLabel,T152C!
,,,,,,,,NoLabel,T16311C!
,,,,,,,,,,,NoLabel,C16311T!!
,,,,,,,,,,NoLabel,T195C!
,,,,,,,,,,NoLabel,T199C
,,,,,,,,,NoLabel,C16234T
,,,,,,,,,NoLabel,T152C!
,,,,,,

In [10]:
it = 'phylotree_17_190121.csv'

In [11]:
ot = 'phylotree_17_190127.csv'

In [12]:
f = open(it,'r')
f = f.readlines()
o = open(ot,'w')

In [13]:
dic_h = {}
c = 0
for branch in f:
	branch = branch.split(',')
	dic_h[c]=''
	for x in xrange(len(branch)):
		if branch[x] == '':
			pass
		else:
			if branch[x] != "NoLabel" and branch[x][0].isupper():
				dic_h[c] = (x,branch[x])
				#print dic_h[c],c,branch
				new_line = map(lambda x:str(x),branch)
				new_line = ','.join(new_line).strip()
				o.write(new_line+'\n')
				break
			elif branch[x] == "NoLabel":
				#print 'Here is the problem',c,x
				z = c-1
				while z > 0:
					#print dic_h[z],z,branch[x],x
					if dic_h[z][0] < x:
						distance = str(c-z)
						#print "distance",distance
						new_label = dic_h[z][1]+"_"+str(branch[x+1]).strip()
						dic_h[c] = (x,new_label)
						branch[x] = new_label
						new_line = map(lambda x:str(x),branch)
						new_line = ','.join(new_line).strip()
						break
						o.write(new_line+'\n')
					else:
						z = z-1

			else:
				pass
	c += 1

In [14]:
o.close()

In [15]:
import getopt, sys, re, os, glob, csv
from classifier import tree, NGclassify, consts, datatypes, parse_mhcs
from bioinf.seqs import SeqList
import io_modules.csv
import io_modules.old_table
import io_modules.serialize
import pandas as pd
import os.path
sys.setrecursionlimit(100000)

In [16]:
csvfile = 'phylotree_17_190127.csv'
out_fname = 'haplogroups'

In [17]:
#write pickle file for MToolBox
tree_file = csv.reader(open(csvfile, 'rb'))

pickle_fname = csvfile + '.pickle'
aplo_list = io_modules.csv.parse_csv(tree_file)
htree = tree.HaplogroupTree(aplo_list=aplo_list)
pickle_file = open(pickle_fname, 'wb')
pickle_file.write(htree.serialize())
pickle_file.close()
#write out alleles and haplogroups defined for HmtDB in csv file
pickle_file = pickle_fname
out_file = out_fname + '.csv'
htree = tree.HaplogroupTree(pickle_data=open(pickle_file, 'rb').read())
fh = csv.writer(open(out_file, 'wb'))
for haplo_name in htree:
	io_modules.old_table.write_haplogroup(fh, '', htree[haplo_name])

#write haplogrups.txt tab delimited file for MToolBox
out_file2 = out_fname + '.txt'
hap_file = pd.read_csv(out_file,sep=',',header=None)
subset = hap_file[[0,1,3]]
subset.dropna(inplace=True)
alleles = subset[1].astype(str).str.cat(subset[3])
subset.insert(1,'Allele',alleles)
subset = subset[[0,'Allele']]
subset.columns=['haplogroup_code','POSITIONnucleotidic_change']
subset.to_csv(out_file2,sep='\t',header=True,index=None)


parent level after,  [None]
level is  0


UnboundLocalError: local variable 'value' referenced before assignment